In [1]:
%%writefile formality_cleaning.py

# Code for converting a csv with JSON column into nice df in pandas
import csv as csv
import pandas as pd
import math
import json
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import datetime as dt



from functools import reduce
from statsmodels.formula.api import ols
from IPython.display import display


#######################################
####### Functs for getting data  ######
#######################################

# convert json columns to dicts 
def decode_json_entry(json_db_entry):
    json_db_entry = json_db_entry.replace('\\"', '').replace('\\', '')
    try:
        decoded_json_entry = json.loads(json_db_entry)
    except json.JSONDecodeError:
        print("Error!",json_db_entry )
    return decoded_json_entry

def get_key(entry, val):
    return 'data_type' in entry and val == entry["data_type"];

def return_key_df(df, key):
    return df[df.data.apply(lambda x: get_key(x, key))]['data'].apply(pd.Series).drop('data_type', axis=1)


############################
####### Get the data  ######
############################

df = pd.read_csv('study_data/CHI2019_submission/litw_iq_data.csv', sep=";", engine='python')
df.data = df.data.apply(decode_json_entry)

df_data = df[df.data.apply(lambda x: get_key(x, 'study:data'))]['data'].apply(pd.Series).drop('data_type', axis=1)

# Collect everything into one df, get some other useful vars
attention_checks_df = df[df.data.apply(lambda x: get_key(x, 'attentionCheck'))]['data'].apply(pd.Series).drop('data_type', axis=1)
formality_survey_df = df[df.data.apply(lambda x: get_key(x, 'formalitySurvey'))]['data'].apply(pd.Series).drop('data_type', axis=1)
formality_level_df = df[df.data.apply(lambda x: get_key(x, 'formalityLevel'))]['data'].apply(pd.Series).drop('data_type', axis=1)
# use only formal/informal version
formality_level_df = formality_level_df[formality_level_df['formality_level'] != 'normal']

values_df = df[df.data.apply(lambda x: get_key(x, 'valueSurvey'))]['data'].apply(pd.Series).drop('data_type', axis=1)
values_df = values_df.rename(index=str, columns={'self-direction': 'self_direction'})

df_correct = df_data.groupby('uuid')['is_correct'].describe()
df_correct = df_correct[df_correct['count'] == 32]
df_correct = df_correct.reset_index().drop(['unique', 'top', 'count'], axis=1)
# rename correct column and drop count col
df_correct = df_correct.rename(index=str, columns={"freq": "score"})

print('length of people who did all 32 rounds (2 practice): ', len(df_correct))


# get drop out column
df_checkpoints = df[df.data.apply(lambda x: get_key(x, 'tracking:checkpoint'))]['data'].apply(pd.Series).drop('data_type', axis=1)
df_checkpoints = df_checkpoints.groupby('uuid')['dropoutCode'].describe().reset_index()[['uuid', 'count']]
df_checkpoints['dropout'] = df_checkpoints['count'] < 41 

# # Getting clickthrough data -- Taken out since not using this right now, will need to download new data
# df_a_b = pd.read_csv('study_data/click_through.csv', engine='python')
# df_a_b.columns = ['id', 'participant_id', 'data', 'timestamp']
# df_a_b.data = df_a_b.data.apply(lambda x: json.loads(x))
# df_a_b['timestamp'] = pd.to_datetime(df_a_b['timestamp'])
# df_a_b = df_a_b.data.apply(pd.Series)
# df_a_b = df_a_b[df_a_b['study_name'] == 'spatial_awareness']

# merge to formality rating
df_formality_dropout = formality_level_df.merge(df_checkpoints, on='uuid')

# get demographics 
df_demographics = df[df.data.apply(lambda x: get_key(x, 'study:demographics'))]['data'].apply(pd.Series).drop('data_type', axis=1)
df_demographics[['education', 'age']] = df_demographics[['education', 'age']].apply(pd.to_numeric)

values = list(values_df.drop('uuid', axis=1).columns)

# convert values to numeric form
values_df[values] = values_df[values].apply(pd.to_numeric)

# formulas for coservation, and self-transcendence, etc.
values_df['conservation'] = .92 + (.15 * values_df['power']) + (.03 * values_df['achievement']) - (.17 * values_df['hedonism']) - (.25 * values_df['stimulation']) - (.31 * values_df['self_direction']) - (.26 * values_df['universalism']) + (.04 * values_df['benevolence']) + (.30 * values_df['tradition']) + (.30 * values_df['conformity']) + (.20 * values_df['security'])
values_df['self_transcendence'] = -.56 - (.30 * values_df['power']) - (.33 * values_df['achievement']) - (.16 * values_df['hedonism']) - (.14 * values_df['stimulation']) + (.04 * values_df['self_direction']) + (.22 * values_df['universalism']) + (.24 * values_df['benevolence']) + (.12 * values_df['tradition']) + (.03 * values_df['conformity']) + (.03 * values_df['security'])

# make comments df
df_comments = return_key_df(df, 'study:comments')

# merge all the dfs
dfs = [df_correct, attention_checks_df, formality_survey_df, formality_level_df, values_df, df_comments]
df_formality_values = reduce(lambda left,right: pd.merge(left,right,on='uuid'), dfs)

# convert values to numeric form
df_formality_values[['formality_rating', 'appropriate_rating', 'score']] = df_formality_values[['formality_rating', 'appropriate_rating', 'score']].apply(pd.to_numeric)

# even though TECHNICALLY true and false are 1/0, the logit function doesn't seem to know that
# so this is a way of fixing that.
df_formality_values.replace(to_replace={'engaged' : {True: '1', False: '0'}}, inplace = True)
df_formality_values['engaged'] = df_formality_values['engaged'].apply(pd.to_numeric)

# same for dropout
df_formality_dropout.replace(to_replace={'dropout' : {True: '1', False: '0'}}, inplace = True)
df_formality_dropout['dropout'] = df_formality_dropout['dropout'].apply(pd.to_numeric)

##############################
####### make a final df ######
##############################
df_dem_formality_values = df_demographics[['age', 'education', 'gender', 'uuid', 'retake', 'country0']].merge(df_formality_values, on='uuid')
# add in english variable
df_dem_formality_values['english'] =  df_dem_formality_values['country0'].apply(lambda x: x in ['United States', 'United Kingdom', 'Canada', 'Australia', 'Ireland', 'New Zealand', 'American Samoa'])

# convert gender to male/female
df_dem_formality_values['gender'] = df_dem_formality_values['gender'].replace({'1':'female', '0':'male'})


##############################
##### Add in turker stuff ####
##############################

# Turkers who started
df_turker = return_key_df(df, 'MTurk')
print("Turkers who started:", len(df_turker))

# Turkers who finished
df_turker_id = return_key_df(df, 'MturkWorkerID')

# Only take those those also submitted through mturk - no
# df_turk_results_id_1 = pd.DataFrame.from_csv('study_data/MTurk_results_1.csv')
# df_turk_results_id_2 = pd.DataFrame.from_csv('study_data/MTurk_results_2.csv')

# df_turk_results_id = pd.concat([df_turk_results_id_1,df_turk_results_id_2])
# df_turk_results_id = df_turk_results_id.rename(index=str, columns={'Answer.surveycode':'MTurkworkerID'})

# df_turker_id = df_turker_id.merge(df_turk_results_id[['MTurkworkerID']], on='MTurkworkerID')

# df_turker_id = df_turker_id.merge(df_turk_results_id, on='MTurkworkerID')
print("Turkers who finished:", len(df_turker_id))

# make new df of turker and formality/values (backwards compatability on the turker notebook)
df_dem_formality_turker = df_turker.merge(df_dem_formality_values, on='uuid', how='right')
df_dem_formality_turker['Turker'] = df_dem_formality_turker['Turker'].fillna(False)

# updated main df with turker info
df_dem_formality_values = df_turker.merge(df_dem_formality_values, on='uuid', how='right')
df_dem_formality_values['Turker'] = df_dem_formality_values['Turker'].fillna(False)

# Fill NaNs with false for both (above and below)

# Same for dropout

df_formality_turker_dropout = df_turker.merge(df_formality_dropout, on='uuid', how='right')
df_formality_turker_dropout['Turker'] = df_formality_turker_dropout['Turker'].fillna(False)


df_formality_dropout = df_turker.merge(df_formality_dropout, on='uuid', how='right')
df_formality_dropout['Turker'] = df_formality_dropout['Turker'].fillna(False)

##############################
##### End Turker Addition ####
##############################

##############################
######## Getting Time ########
##############################

# Get end of study (comments)
comments = df[df.data.apply(lambda x: get_key(x, 'study:comments'))]
checkpoints = df[df.data.apply(lambda x: get_key(x, 'tracking:checkpoint'))]

# Get beginning of study (IRB)
irb_checkpoint = checkpoints[checkpoints.data.apply(lambda x: x['description'] == 'irb')]

# Get uuid as own column
comments['uuid'] = comments.data.apply(pd.Series)['uuid']
irb_checkpoint['uuid'] = irb_checkpoint.data.apply(pd.Series)['uuid']

# convert the time stamp on both
comments['timestamp'] = pd.to_datetime(comments['timestamp'])
irb_checkpoint['timestamp'] = pd.to_datetime(irb_checkpoint['timestamp'])

# make a time dataframe with length of study
time_df = pd.merge(comments, irb_checkpoint, on='uuid')
time_df['time_diff'] = time_df['timestamp_x'] - time_df['timestamp_y']

# add time to take test to main df
df_dem_formality_values = time_df[['uuid', 'time_diff']].merge(df_dem_formality_values, on='uuid')
df_dem_formality_values['time_diff'] = df_dem_formality_values['time_diff'].apply(lambda x: (x.total_seconds())/60)

##############################
######### End Time ########### 
##############################


##############################
########## Cleaning ##########
##############################

print('Before cheating removed:', len(df_dem_formality_values))
df_dem_formality_values = df_dem_formality_values[df_dem_formality_values.apply(lambda x: x['cheating'] != 'yes', axis=1)]
print('After cheating removed:', len(df_dem_formality_values))

df_dem_formality_values = df_dem_formality_values[df_dem_formality_values.apply(lambda x: x['technical'] != 'yes', axis=1)]
print('After technical removed:', len(df_dem_formality_values))

print('Before retake and annoying education person:', len(df_dem_formality_values))
df_dem_formality_values = df_dem_formality_values[df_dem_formality_values['retake'] == '0'] 
df_dem_formality_values = df_dem_formality_values[df_dem_formality_values['education'] != 666]
print('After retake and annoying education person::', len(df_dem_formality_values))

print('Before time:', len(df_dem_formality_values))
df_dem_formality_values = df_dem_formality_values[df_dem_formality_values['time_diff'] < ((dt.timedelta(minutes=60)).total_seconds()/60)]
print('after time:', len(df_dem_formality_values))

print('Before dem drop:', len(df_dem_formality_values))
df_dem_formality_values_no_drop = df_dem_formality_values.copy()
df_dem_formality_values = df_dem_formality_values.dropna(subset=['gender', 'age', 'education'])
df_dem_formality_values = df_dem_formality_values[df_dem_formality_values['gender'] != '2']
print('after dem drop:', len(df_dem_formality_values))

print('Done cleaning formality data')
print('Total number of full participants (completed up through comments):',
      len(df_dem_formality_values))

##############################
##### Done Cleaning ##########
##############################


##############################
########## Funcs #############
##############################

# simple function for not repeating lines of code for computing and displaying linear regression results
def run_lin_regr(df, formula):
    print("Running linear regression for:", formula)
    model = ols(formula, data=df)
    results = model.fit()
    display(results.summary())

def run_mixed_regr(df, formula, g):
    print("Running mixed regression for:", formula)
    model = smf.mixedlm(formula, df, groups=g)
    
    results = model.fit()
    display(results.summary())
    
# useful for checking all values at once
value_formula = ''.join([v+' + ' for v in values]).strip(' + ')




Overwriting formality_cleaning.py


16